In [14]:
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


from selenium import webdriver #selenium 4.20.0
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import WebDriverException

from webdriver_manager.chrome import ChromeDriverManager
 # version 4.0.1

In [2]:
options = webdriver.ChromeOptions()
options.set_capability(
    "goog:loggingPrefs", {"performance": "ALL", "browser": "ALL"}
)

# Make sure you already have Chrome installed
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [36]:
driver.set_page_load_timeout(30)

try:
    driver.get("https://www.sofascore.com/api/v1/event/11873967/player/856714/heatmap")
except:
    pass


driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [37]:
# extract requests from logs
logs_raw = driver.get_log("performance")
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

In [38]:
desired_keywords = ['heatmap']

for log in logs:
    # Check if the log contains the 'method' key
    if 'method' in log:
        # Check if the log method is 'Network.responseReceived'
        if log["method"] == "Network.responseReceived":
            # Extract request ID and URL
            request_id = log["params"]["requestId"]
            url = log["params"]["response"]["url"]
            # Check if the URL contains any of the desired keywords
            if any(keyword in url for keyword in desired_keywords):
                # Print the request ID and URL
                print(f"Request ID: {request_id}, URL: {url}")
                # # Try to retrieve the response body using the request ID
                try:
                    response = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': request_id})
                    stats = response['body']
                    if stats:
                        print("Response body successfully retrieved:")
                        print(stats)
                        break
                    else:
                        print("Response body is empty. Trying next request ID...")
                except Exception as e:
                    print(f"Error occurred while retrieving response for request ID {request_id}: {e}")
                    continue

stats


Request ID: 1908.3019, URL: https://www.sofascore.com/api/v1/event/11873967/player/886930/heatmap
Error occurred while retrieving response for request ID 1908.3019: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No resource with given identifier found"}
  (Session info: chrome=126.0.6478.116)
Stacktrace:
	GetHandleVerifier [0x00DDC1C3+27395]
	(No symbol) [0x00D73DC4]
	(No symbol) [0x00C71B7F]
	(No symbol) [0x00C5CFED]
	(No symbol) [0x00C5BC73]
	(No symbol) [0x00C5C3CD]
	(No symbol) [0x00C5C368]
	(No symbol) [0x00C748B4]
	(No symbol) [0x00CF7431]
	(No symbol) [0x00CD399C]
	(No symbol) [0x00CECB24]
	(No symbol) [0x00CD3736]
	(No symbol) [0x00CA7541]
	(No symbol) [0x00CA80BD]
	GetHandleVerifier [0x01093A93+2876371]
	GetHandleVerifier [0x010E7F5D+3221661]
	GetHandleVerifier [0x00E5D634+556916]
	GetHandleVerifier [0x00E6474C+585868]
	(No symbol) [0x00D7CE04]
	(No symbol) [0x00D79818]
	(No symbol) [0x00D799B7]
	(No symbol) [0x00D6BF0E]
	BaseThreadInitThunk [0x76

'{"heatmap":[{"x":28,"y":59},{"x":33,"y":60},{"x":71,"y":87},{"x":72,"y":88},{"x":72,"y":88},{"x":24,"y":33},{"x":49,"y":59},{"x":43,"y":40},{"x":77,"y":83},{"x":67,"y":71},{"x":11,"y":66},{"x":57,"y":29},{"x":60,"y":58},{"x":34,"y":72},{"x":32,"y":81},{"x":25,"y":52},{"x":36,"y":60},{"x":66,"y":35},{"x":76,"y":66},{"x":39,"y":87},{"x":66,"y":77},{"x":66,"y":77},{"x":61,"y":56},{"x":72,"y":41},{"x":70,"y":33},{"x":88,"y":82},{"x":87,"y":85},{"x":30,"y":60},{"x":60,"y":67},{"x":82,"y":78},{"x":28,"y":89},{"x":28,"y":89},{"x":37,"y":14},{"x":39,"y":26},{"x":64,"y":31},{"x":66,"y":8},{"x":90,"y":71},{"x":61,"y":48},{"x":61,"y":49},{"x":54,"y":45},{"x":66,"y":81},{"x":70,"y":75},{"x":62,"y":64},{"x":48,"y":89},{"x":48,"y":89},{"x":32,"y":75},{"x":32,"y":72},{"x":34,"y":68},{"x":38,"y":75},{"x":37,"y":63},{"x":46,"y":79},{"x":64,"y":49},{"x":64,"y":53},{"x":65,"y":54},{"x":62,"y":81},{"x":72,"y":80},{"x":88,"y":94},{"x":69,"y":86},{"x":67,"y":86},{"x":71,"y":66},{"x":78,"y":59},{"x":74,"y":

In [43]:
data = json.loads(stats)
df = pd.DataFrame(data['heatmap'])
df

,x,y
0,28,59
1,33,60
2,71,87
3,72,88
4,72,88
...,...,...
76,57,71
77,52,75
78,24,70
79,24,29


In [44]:
df.to_csv('heatmap.csv', index=False)